<a href="https://colab.research.google.com/github/itsshuvra/Animal-Image-Classification/blob/main/AnimalClassificationKNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Image Preprocessing**

In [ ]:
#necessary packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import os

In [ ]:
#necessary package
import cv2

class ImgProcessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        self.width=width
        self.height=height
        self.inter=inter

    def process(self, img):
        #same image ratio for all images
        return cv2.resize(img, (self.width, self.height), interpolation=self.inter)


In [ ]:
#nessary package
import numpy as np

class DataLoader:
    def __init__(self, prepros=None):
        self.prepros=prepros
        if self.prepros is None:
            self.prepros=[]

    def load(self, imgpaths, verbose=-1):
        #create two list for data and label
        data = []
        labels=[]
        #loop over the input image
        for(i, imgpath) in enumerate(imgpaths):
            img=cv2.imread(imgpath)
            label=imgpath.split(os.path.sep)[-2]

            if self.prepros is not None:
                for p in self.prepros:
                    img=p.process(img)

            data.append(img)
            labels.append(label)

            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1, len(imgpaths)))
        return (np.array(data), np.array(labels))

## **KNN**

In [ ]:
#dataset path
dataset_path = '/content/drive/MyDrive/animals/'
neighbors = 3
jobs = 1

In [ ]:
print("Loading Images:")
imgpaths=list(paths.list_images(dataset_path))
# 32*32 pixel image size processing
ip = ImgProcessor(32, 32)
dl = DataLoader(prepros=[ip])
(data, labels) = dl.load(imgpaths, verbose=500)
data = data.reshape((data.shape[0], 3072))
print("[INFO] features matrix: {:.1f}KB".format(data.nbytes / (1024)))
print(data[1].shape)
# encoding the labels for each animal
# convert these to interger like 0, 1, 2
# for cat class 0, dog class 1 and panda class 2

le = LabelEncoder()
labels = le.fit_transform(labels)

#75% training and 25% testing data
(Xtrain, Xtest, ytrain, ytest) = train_test_split(data, labels, test_size=0.25, random_state=40)

#evaluating with knn classifier
model = KNeighborsClassifier(n_neighbors=neighbors, n_jobs=jobs)

model.fit(Xtrain, ytrain)


print(classification_report(ytest, model.predict(Xtest), target_names=le.classes_))

Loading Images:
[INFO] processed 500/3000
[INFO] processed 1000/3000
[INFO] processed 1500/3000
[INFO] processed 2000/3000
[INFO] processed 2500/3000
[INFO] processed 3000/3000
okkkkkkk
[INFO] features matrix: 9000.0KB
(3072,)
              precision    recall  f1-score   support

        cats       0.39      0.62      0.48       253
        dogs       0.42      0.44      0.43       252
       panda       0.89      0.32      0.47       245

    accuracy                           0.46       750
   macro avg       0.57      0.46      0.46       750
weighted avg       0.57      0.46      0.46       750

